## Домашнее задание 2: Линейные модели. Работа с признаками

Правила:

* Домашнее задание оценивается в 10 баллов.

* Можно использовать без доказательства любые результаты, встречавшиеся на лекциях или семинарах по курсу, если получение этих результатов не является вопросом задания.

* Можно использовать любые свободные источники с *обязательным* указанием ссылки на них.

* Плагиат не допускается.

* Старайтесь сделать код как можно более оптимальным и читаемым.

In [159]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%pylab inline

Populating the interactive namespace from numpy and matplotlib


/Users/elenaefimova/opt/anaconda3/lib/python3.8/site-packages/IPython/core/magics/pylab.py:159: UserWarning: pylab import has clobbered these variables: ['copy']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


В этом задании мы рассмотрим различные аспекты построения линейной модели. Мы будем работать с одним из классических наборов данных в статистике, содержащим информацию о бриллиантах. Описание можно посмотреть [здесь](https://www.kaggle.com/shivam2503/diamonds).

In [160]:
data = pd.read_csv('diamonds.csv')
data.head(5)

,Unnamed: 0,carat,cut,color,clarity,depth,table,price,x,y,z
0,1,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,2,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,3,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,4,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,5,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


Мы будем решать задачу предсказания цены бриллианта `price` в зависимости от его характеристик.

## Построение модели

**Задание 1 (1 балл)** Есть ли в наборе данных пропущенные значения? Если да, удалите их. 

In [161]:
data.isnull().sum()

Unnamed: 0    0
carat         0
cut           0
color         0
clarity       0
depth         0
table         0
price         0
x             0
y             0
z             0
dtype: int64

**Задача 2 (1 балл)** Есть ли в наборе данных бессмысленные столбцы (признаки, не несущие дополнительной информации)? Если да, то удалите их. Поясните свой выбор (напишите текстом обоснование).

In [162]:
# не несет доп информации столбец Unnamed: 0. Он фактически повторяет индекс
data = data.drop(['Unnamed: 0'], axis=1)
data.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


**Задание 3 (1 балл)** Линейная регрессия основана на предположении о линейной связи между признаками и целевой переменной, а потому перед выбором переменных для включения в модель имеет смысл проверить, насколько эта связь выполняется. Для следующих пунктов нам также потребуются корреляции между признаками. Выведите матрицу корреляций между всеми вещественными признаками и целевой переменной.

Какие вещественные признаки коррелируют с целевой переменной больше всего?

In [163]:
# введем целевую переменную - цена
y = data['price'] # целевая переменная
numericData = data._get_numeric_data()
numericData.corrwith(y)

carat    0.921591
depth   -0.010647
table    0.127134
price    1.000000
x        0.884435
y        0.865421
z        0.861249
dtype: float64

In [164]:
numericData.corr()

,carat,depth,table,price,x,y,z
carat,1.000000,0.028224,0.181618,0.921591,0.975094,0.951722,0.953387
depth,0.028224,1.000000,-0.295779,-0.010647,-0.025289,-0.029341,0.094924
table,0.181618,-0.295779,1.000000,0.127134,0.195344,0.183760,0.150929
price,0.921591,-0.010647,0.127134,1.000000,0.884435,0.865421,0.861249
x,0.975094,-0.025289,0.195344,0.884435,1.000000,0.974701,0.970772
y,0.951722,-0.029341,0.183760,0.865421,0.974701,1.000000,0.952006
z,0.953387,0.094924,0.150929,0.861249,0.970772,0.952006,1.000000


In [165]:
# коррелируют carat,x,y,x

**Задание 4 (1 балл)** Так как линейная модель складывает значения признаков с некоторыми весами, нам нужно аккуратно обработать категориальные признаки. Закодируйте категориальные переменные при помощи OneHot-кодирования (pd.get_dummies). Не забудьте поставить значение параметра drop_first равным True.

In [166]:
data = pd.get_dummies(data, columns=['cut', 'color', 'clarity'], drop_first=True)
data.head()

,carat,depth,table,price,x,y,z,cut_Good,cut_Ideal,cut_Premium,...,color_H,color_I,color_J,clarity_IF,clarity_SI1,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2
0,0.23,61.5,55.0,326,3.95,3.98,2.43,0,1,0,...,0,0,0,0,0,1,0,0,0,0
1,0.21,59.8,61.0,326,3.89,3.84,2.31,0,0,1,...,0,0,0,0,1,0,0,0,0,0
2,0.23,56.9,65.0,327,4.05,4.07,2.31,1,0,0,...,0,0,0,0,0,0,1,0,0,0
3,0.29,62.4,58.0,334,4.20,4.23,2.63,0,0,1,...,0,1,0,0,0,0,0,1,0,0
4,0.31,63.3,58.0,335,4.34,4.35,2.75,1,0,0,...,0,0,1,0,0,1,0,0,0,0


**Задание 5 (1 балл)** 
Создайте матрицу X, содержащую все признаки, и не содержащую целевую переменную price. Также создайте вектор y, содержащий целевую переменную price.

In [167]:
y = data['price']
X = data._get_numeric_data().drop('price', axis=1)

**Задание 5 (1 балл)** 
Перемешайте данные! 

Разделите выборку на тренировочную и тестовую. Долю тестовой выборки укажите равной 0.3.

In [169]:
from sklearn.model_selection import train_test_split

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.3)

**Задание 6 (1 балл)** Зачастую при использовании линейных моделей вещественные признаки масштабируются.  В этой задаче масштабируйте вещественные признаки тренировочной и тестовой выборок при помощи модуля `StandardScaler`.

После применения масштабирования матрица перестает быть объектом Pandas Dataframe - решите эту проблему.

In [170]:
from sklearn.preprocessing import StandardScaler

# fit calculates the parameters (e.g. 𝜇 and 𝜎 in case of StandardScaler) and saves them as an internal object's state
# transform() method to apply the transformation to any particular set of examples
# fit_transform() joins these two steps and is used for the initial fitting of parameters on the training set 𝑥, while also returning the transformed  𝑥

# your code here
ss = StandardScaler()
scaler = ss.fit(Xtrain)
Xtrain2 = scaler.fit_transform(Xtrain)
Xtest2 = scaler.transform(Xtest)

Xtrain2 = pd.DataFrame(Xtrain2,columns = Xtrain.columns)
Xtest2 = pd.DataFrame(Xtest2,columns = Xtest.columns)
Xtest2.head()

,carat,depth,table,x,y,z,cut_Good,cut_Ideal,cut_Premium,cut_Very Good,...,color_H,color_I,color_J,clarity_IF,clarity_SI1,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2
0,0.217040,0.035999,0.250142,0.384533,0.355539,0.378460,-0.314576,-0.818968,1.705015,-0.535642,...,-0.425754,-0.333392,4.270228,-0.184194,1.758443,-0.455122,-0.421972,-0.540348,-0.268102,-0.321944
1,0.217040,0.662117,1.147803,0.357805,0.293140,0.407207,-0.314576,-0.818968,1.705015,-0.535642,...,-0.425754,-0.333392,-0.234180,-0.184194,-0.568685,-0.455122,2.369828,-0.540348,-0.268102,-0.321944
2,0.217040,0.314274,1.147803,0.331077,0.382281,0.392833,-0.314576,-0.818968,1.705015,-0.535642,...,-0.425754,-0.333392,-0.234180,-0.184194,1.758443,-0.455122,-0.421972,-0.540348,-0.268102,-0.321944
3,0.870880,0.175136,-1.096349,0.954729,1.024093,1.010899,-0.314576,1.221049,-0.586505,-0.535642,...,-0.425754,-0.333392,-0.234180,-0.184194,-0.568685,-0.455122,-0.421972,-0.540348,3.729923,-0.321944
4,-1.027365,1.288236,-0.198688,-1.263690,-1.293562,-1.159517,3.178883,-0.818968,-0.586505,-0.535642,...,2.348774,-0.333392,-0.234180,-0.184194,1.758443,-0.455122,-0.421972,-0.540348,-0.268102,-0.321944


**Задание 7 (1 балл)** Обучите линейную регрессию на тренировочной выборке. Выведите r2-score на тренировочной и тестовой выборках.

In [171]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

# your code here
lr = LinearRegression()
lr.fit(Xtrain2, ytrain)

train_pred = lr.predict(Xtrain2)
test_pred = lr.predict(Xtest2)

print("Train MSE: {}".format(np.sqrt(mean_squared_error(ytrain, train_pred))))
print("Test MSE: {}".format(np.sqrt(mean_squared_error(ytest, test_pred))))

print(f'''
train R2={r2_score(ytrain, train_pred)}
''')

print(f'''
test R2={r2_score(ytest, test_pred)}
''')

Train MSE: 1116.3437611555535
Test MSE: 1161.8515713846298

train R2=0.922112029540476


test R2=0.9141135973121071



**Задание 8 (1 балл)** Выведите на экран веса, которые линейная регрессия присвоила признакам. Назовите вещественные переменные, оценки коэффициентов которых по модулю на порядок превышают оценки прочих вещественных переменных.

In [172]:
# The coefficients
print('Coefficients: \n', lr.coef_, "\n")
print('Feature:  Weight:')

for feature, coeff in zip(data.columns,lr.coef_.T):
    print(feature, coeff)
    
# carat, price, color_J, clarity_***

Coefficients: 
 [ 5366.50067673   -90.76685885   -62.37976104 -1162.36845148
    77.71784489   -89.02951647   165.29847251   414.24899703
   332.7162927    306.60552967   -84.49424436  -105.68405884
  -198.93933262  -355.71232252  -439.5515694   -526.60658738
   928.39539695  1538.25508142   983.17547926  1605.25129121
  1751.42101009  1233.71780778  1418.82120343] 

Feature:  Weight:
carat 5366.500676727457
depth -90.76685885080514
table -62.379761036982565
price -1162.3684514785195
x 77.71784488937298
y -89.02951647494316
z 165.2984725054568
cut_Good 414.248997032679
cut_Ideal 332.71629269579483
cut_Premium 306.60552967092445
cut_Very Good -84.49424436491584
color_E -105.68405883612732
color_F -198.93933261610357
color_G -355.7123225238104
color_H -439.55156939770313
color_I -526.6065873753996
color_J 928.3953969454618
clarity_IF 1538.2550814173244
clarity_SI1 983.1754792580304
clarity_SI2 1605.2512912111242
clarity_VS1 1751.4210100919254
clarity_VS2 1233.7178077814303
clarity_VVS1 1

## Попытка улучшить качество модели

Эта часть является творческой, и задания в ней немного сложнее, чем предыдущие. Поэтому за каждое из них можно получить до двух баллов.

**Задание 9* (2 балла)** Как можно заметить из анализа корреляционной матрицы в задании 3, между некоторыми признаками имеется сильная корреляция, что может быть индикатором проблемы *мультиколлинеарности*. Различия в порядке коэффициентов, выявленные в предыдущей задаче, также свидетельствуют об этом. Для решения этой проблемы можно либо исключить некоторые признаки из модели (например, если признак линейно зависим с какими-то другими, его можно исключить из модели, т.е. удалить из матрицы объект-признак и заново обучить модель).

Удалите из матриц Xtrain и Xtest признак, который наиболее сильно коррелирует с остальными. Заново обучите модель и оцените её качество. Улучшилось ли качество модели?
Попробуйте удалить какой-то другой признак (можете попробовать несколько вариантов). Помогло ли это улучшить качество модели?

In [173]:
# попробуем удалить три признака, которые сильно коррелируют с другими
X1 = X.drop(['x', 'y', 'z'], axis=1)

In [174]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
Xtrain, Xtest, ytrain, ytest = train_test_split(X1, y, test_size=0.3)

ss = StandardScaler()
scaler = ss.fit(Xtrain)
Xtrain2 = scaler.fit_transform(Xtrain)
Xtest2 = scaler.transform(Xtest)

Xtrain2 = pd.DataFrame(Xtrain2,columns = Xtrain.columns)
Xtest2 = pd.DataFrame(Xtest2,columns = Xtest.columns)
Xtest2.head()

,carat,depth,table,cut_Good,cut_Ideal,cut_Premium,cut_Very Good,color_E,color_F,color_G,color_H,color_I,color_J,clarity_IF,clarity_SI1,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2
0,0.275919,-1.009189,0.244396,-0.315033,-0.813383,1.694411,-0.536174,-0.472701,-0.462903,-0.513555,-0.425754,-0.334029,4.247452,-0.185726,-0.563627,-0.45317,2.376454,-0.543047,-0.271491,-0.322895
1,1.493743,-0.244452,0.244396,-0.315033,-0.813383,-0.590176,1.865066,-0.472701,2.160282,-0.513555,-0.425754,-0.334029,-0.235435,-0.185726,-0.563627,-0.45317,-0.420795,-0.543047,-0.271491,3.096983
2,-1.025893,0.937414,0.244396,-0.315033,-0.813383,-0.590176,1.865066,-0.472701,-0.462903,1.947209,-0.425754,-0.334029,-0.235435,-0.185726,-0.563627,-0.45317,-0.420795,1.841460,-0.271491,-0.322895
3,0.212928,-1.009189,-0.648769,-0.315033,-0.813383,1.694411,-0.536174,2.115502,-0.462903,-0.513555,-0.425754,-0.334029,-0.235435,-0.185726,1.774222,-0.45317,-0.420795,-0.543047,-0.271491,-0.322895
4,0.779846,-0.870146,-0.202186,-0.315033,-0.813383,-0.590176,1.865066,-0.472701,-0.462903,-0.513555,-0.425754,-0.334029,-0.235435,-0.185726,-0.563627,-0.45317,2.376454,-0.543047,-0.271491,-0.322895


In [175]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

# your code here
lr = LinearRegression()
lr.fit(Xtrain2, ytrain)

train_pred = lr.predict(Xtrain2)
test_pred = lr.predict(Xtest2)
print("Train MSE: {}".format(np.sqrt(mean_squared_error(ytrain, train_pred))))
print("Test MSE: {}".format(np.sqrt(mean_squared_error(ytest, test_pred))))

print(f'''
train R2={r2_score(ytrain, train_pred)}
''')

print(f'''
test R2={r2_score(ytest, test_pred)}
''')

# качество модели ухудшилось, но не намного

Train MSE: 1157.5516591816718
Test MSE: 1152.1058545004962

train R2=0.9166961299891525


test R2=0.9144722435125905



In [176]:
# попробуем удалить другой признак
X2 = X.drop(['carat'], axis=1)

In [177]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
Xtrain, Xtest, ytrain, ytest = train_test_split(X2, y, test_size=0.3)

ss = StandardScaler()
scaler = ss.fit(Xtrain)
Xtrain2 = scaler.fit_transform(Xtrain)
Xtest2 = scaler.transform(Xtest)

Xtrain2 = pd.DataFrame(Xtrain2,columns = Xtrain.columns)
Xtest2 = pd.DataFrame(Xtest2,columns = Xtest.columns)

In [178]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

# your code here
lr = LinearRegression()
lr.fit(Xtrain2, ytrain)

train_pred = lr.predict(Xtrain2)
test_pred = lr.predict(Xtest2)

print("Train MSE: {}".format(np.sqrt(mean_squared_error(ytrain, train_pred))))
print("Test MSE: {}".format(np.sqrt(mean_squared_error(ytest, test_pred))))

print(f'''
train R2={r2_score(ytrain, train_pred)}
''')

print(f'''
test R2={r2_score(ytest, test_pred)}
''')
# нельзя удалять данный признак, тк модель стала работать намного хуже

Train MSE: 1595.075337626729
Test MSE: 1596.5658290838776

train R2=0.8403710762790253


test R2=0.8392866716132239



**Задание 10* (2 балла)** Иногда генерация новых признаков помогает модели лучше находить взаимосвязи между целевой переменной и признаками. Попробуйте придумать новые признаки и добавить их в модель (можно черпать идеи из ноутбука занятия 3). Помогло ли это улучшить качество модели?

In [205]:
import copy

df = X.loc[:, ['carat', 'depth', 'table']]
cols = copy.deepcopy(df.columns)

for col1 in cols:
    for col2 in cols:
        col_name = col1 + '_' + col2
        if col_name not in df.columns:
            df[col_name] = df[col1]*df[col2]
df = df.drop(['carat', 'depth', 'table'], axis=1)
df.columns

Index(['carat_carat', 'carat_depth', 'carat_table', 'depth_carat',
       'depth_depth', 'depth_table', 'table_carat', 'table_depth',
       'table_table'],
      dtype='object')

In [207]:
df1 = pd.concat([X, df], axis=1)
df1

,carat,depth,table,x,y,z,cut_Good,cut_Ideal,cut_Premium,cut_Very Good,...,clarity_VVS2,carat_carat,carat_depth,carat_table,depth_carat,depth_depth,depth_table,table_carat,table_depth,table_table
0,0.23,61.5,55.0,3.95,3.98,2.43,0,1,0,0,...,0,0.0529,14.145,12.65,14.145,3782.25,3382.5,12.65,3382.5,3025.0
1,0.21,59.8,61.0,3.89,3.84,2.31,0,0,1,0,...,0,0.0441,12.558,12.81,12.558,3576.04,3647.8,12.81,3647.8,3721.0
2,0.23,56.9,65.0,4.05,4.07,2.31,1,0,0,0,...,0,0.0529,13.087,14.95,13.087,3237.61,3698.5,14.95,3698.5,4225.0
3,0.29,62.4,58.0,4.20,4.23,2.63,0,0,1,0,...,0,0.0841,18.096,16.82,18.096,3893.76,3619.2,16.82,3619.2,3364.0
4,0.31,63.3,58.0,4.34,4.35,2.75,1,0,0,0,...,0,0.0961,19.623,17.98,19.623,4006.89,3671.4,17.98,3671.4,3364.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53935,0.72,60.8,57.0,5.75,5.76,3.50,0,1,0,0,...,0,0.5184,43.776,41.04,43.776,3696.64,3465.6,41.04,3465.6,3249.0
53936,0.72,63.1,55.0,5.69,5.75,3.61,1,0,0,0,...,0,0.5184,45.432,39.60,45.432,3981.61,3470.5,39.60,3470.5,3025.0
53937,0.70,62.8,60.0,5.66,5.68,3.56,0,0,0,1,...,0,0.4900,43.960,42.00,43.960,3943.84,3768.0,42.00,3768.0,3600.0
53938,0.86,61.0,58.0,6.15,6.12,3.74,0,0,1,0,...,0,0.7396,52.460,49.88,52.460,3721.00,3538.0,49.88,3538.0,3364.0


In [208]:
print(df1.shape)
print(y.shape)

(53940, 32)
(53940,)


In [210]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
Xtrain, Xtest, ytrain, ytest = train_test_split(df1, y, test_size=0.3)

ss = StandardScaler()
scaler = ss.fit(Xtrain)
Xtrain2 = scaler.fit_transform(Xtrain)
Xtest2 = scaler.transform(Xtest)

Xtrain2 = pd.DataFrame(Xtrain2,columns = Xtrain.columns)
Xtest2 = pd.DataFrame(Xtest2,columns = Xtest.columns)


In [211]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

# your code here
lr = LinearRegression()
lr.fit(Xtrain2, ytrain)

train_pred = lr.predict(Xtrain2)
test_pred = lr.predict(Xtest2)

print("Train MSE: {}".format(np.sqrt(mean_squared_error(ytrain, train_pred))))
print("Test MSE: {}".format(np.sqrt(mean_squared_error(ytest, test_pred))))

print(f'''
train R2={r2_score(ytrain, train_pred)}
''')

print(f'''
test R2={r2_score(ytest, test_pred)}
''')

Train MSE: 1104.7651386054745
Test MSE: 1088.4028288147933

train R2=0.9235858747060604


test R2=0.9249407961204537



## Выводы

Сделайте выводы, исходя из проделанной работы.

Какого наилучшего качества удалось добиться? Хорошее ли это качество на ваш взгляд? Что для этого вам пришлось сделать?

Также (по желанию) напишите, была ли эта домашняя работа для вас интересной.

In [ ]:
удалось улучшить качество модели test R2=0.92 и уменьшить ошибку Test MSE: 1088
Для этого я добавила новые признаки, перемножив их, для 3 некоррелирующих признаков carat depth table
Работа была интересной, местами немного сложноватой, но постаралась разобраться